In [48]:
import mbo_utilities.lazy_array
!uv pip install jupyterlab-vim

Sourcing .zshenv...
Using Python 3.12.10 environment at: /home/flynn/repos/work/mbo_utilities/.venv
Audited 1 package in 12ms


In [ ]:
%load_ext autoreload
%autoreload 2

import mbo_utilities as mbo
import fastplotlib as fpl
import numpy as np
from pathlib import Path

In [ ]:
fname_scan = [x for x in Path(r"/home/flynn/lbm_data/raw").glob("*tif*")]
raw_scan = mbo.read_scan(fname_scan)

In [ ]:
raw_scan = mbo.lazy_array.imread("/home/flynn/lbm_data/raw")

In [ ]:
import os

os.environ["MBO_DEBUG"] = "1"

In [ ]:
raw_scan.fix_phase = False
raw_scan.roi = 0

In [ ]:
type(raw_scan)

In [ ]:
%%timeit
raw_scan[:200, 0, :, :]

In [ ]:
raw_scan.data.fix_phase = False

In [ ]:
%%timeit
raw_scan[:200, 0, :, :]

In [ ]:
%%timeit
raw_scan.fix_phase=False
raw_scan[:200, 0, :, :]

In [ ]:
%%timeit
raw_scan.fix_phase=False
raw_scan[:2000, 0, :, :]

In [ ]:
%%timeit
raw_scan.fix_phase=False
raw_scan[::5, 0, :, :]

In [ ]:
%%timeit
raw_scan.fix_phase=False
raw_scan.roi=0
raw_scan[::5, 0, :, :]

In [ ]:
zstore = raw_scan.as_zarr()

## Raw vs Kerchunked

In [ ]:
import tifffile

In [ ]:
files = [x for x in Path("/home/flynn/lbm_data/raw/").glob("*.tif*")]
files

In [ ]:
data = np.concat([tifffile.imread(files[i]) for i in range(len(files))], axis=0)
data.shape

In [ ]:
# /// script
# requires-python = ">=3.13"
# dependencies = [
#     "numpy",
#     "mbo_utilities",
#     "fastplotlib",
# ]
#
# [tool.uv.sources]
# mbo_utilities = { git = "https://github.com/MillerBrainObservatory/mbo_utilities", branch = "dev" }

%load_ext autoreload
%autoreload 2
import time
from functools import partial
from typing import Literal

import tifffile
from pathlib import Path
import tifffile as tiff
import numpy as np

import fastplotlib as fpl
from fastplotlib.ui import EdgeWindow
from fastplotlib.widgets import ImageWidget
from scipy.ndimage import fourier_shift

from mbo_utilities.lazy_array import imread, imwrite
import mbo_utilities as mbo
from mbo_utilities import is_raw_scanimage
from mbo_utilities.graphics._imgui import ndim_to_frame
from mbo_utilities.metadata import has_mbo_metadata

from imgui_bundle import imgui

In [ ]:
def find_si_rois(file):
    """
    Find the ROIs in the current ScanImage session.

    Returns
    -------
    list
        List of ROI names.
    """
    with tifffile.TiffFile(file, mode="r") as _tf:
        if is_raw_scanimage(file):
            si_metadata = _tf.scanimage_metadata
        if has_mbo_metadata(file):
            si_metadata = _tf.shaped_metadata[0]["si"]
        rois = si_metadata["RoiGroups"]["imagingRoiGroup"]["rois"]
    return rois


def write_u16(infile: str | Path, outfile: str | Path):
    img = tiff.imread(infile).astype(np.int32)
    off  = img.min()
    rng  = img.max() - off
    u16  = (img - off).astype(np.uint16)

    tiff.imwrite(
        outfile,
        u16,
        photometric="minisblack",
        bitspersample=16,
        extratags=[
            (340, "H", 1, (0,),   False),
            (341, "H", 1, (rng if rng < 65536 else 65535,), False),
            (65535, "d", 2, (float(off), float(rng)), False)
        ],
    )

def timeit(func):
    """
    Decorator to time a function.
    """
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Function {func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

import matplotlib.pyplot as plt
Path("/tmp/01").mkdir(exist_ok=True,)

data = imread(
    r"/home/flynn/lbm_data/raw"
)

In [ ]:
data.output_xslices, data.output_yslices, data.xslices, data.yslices

In [ ]:
data.output_xslices[0], data.output_xslices[1]

In [ ]:
data.fix_phase = True
data.roi = 2

In [ ]:
iw = fpl.ImageWidget(data, figure_kwargs={"size": (550, 550)})
iw.show()

In [ ]:
zarray = mbo_utilities.lazy_array.ZarrScanView(
    data.data.as_zarr(),
    ys=data.yslices,
    xs=data.xslices,
    oys=data.output_yslices,
    oxs=data.output_xslices,
    roi=None,
)


In [ ]:
iw.close()

In [ ]:
zarray = mbo_utilities.lazy_array.ZarrScanView(
    data.data.as_zarr(),
    yslices=data.yslices,
    xslices=data.xslices,
    roi=0,
)

In [ ]:
zarray.shape, zarray[0, 0, ...].shape

In [ ]:
files = [x for x in Path("/tmp/01/fsspec").glob("*.tif*")]
check = tifffile.imread(files[0])
fpl.ImageWidget(check,
                histogram_widget=True,
                figure_kwargs={"size": (800, 1000),},
                graphic_kwargs={"vmin": check.min(), "vmax": check.max()},
                window_funcs={"t": (np.mean, 0)},
               ).show()